In [83]:
from twitchAPI.twitch import Twitch
from twitchAPI.oauth import UserAuthenticator
from twitchAPI.types import AuthScope
from twitchAPI.helper import first
import random
from datetime import datetime, timezone, timedelta

In [93]:
twitch = await Twitch(client_id, client_secret)
target_scope = [AuthScope.MODERATOR_READ_CHATTERS, AuthScope.MODERATOR_READ_FOLLOWERS, AuthScope.CHANNEL_MANAGE_PREDICTIONS]
auth = UserAuthenticator(twitch, target_scope, force_verify=True)
token, refresh_token = await auth.authenticate()

In [94]:
await twitch.set_user_authentication(token, target_scope, refresh_token)

In [95]:
streamer = await first(twitch.get_users(logins=streamer_name))
streamer_id = streamer.id

In [96]:
chatters = await twitch.get_chatters(streamer_id, streamer_id)
chatters.data

In [99]:
winner = random.choice(chatters.data)
winner_res = await first(twitch.get_users(logins=winner.user_name))
winner_id = winner_res.id
winner.user_name

'L0rdEv'

In [100]:
followers = await twitch.get_channel_followers(streamer_id, user_id=winner_id)

In [102]:
if not followers.data:
    follow_time = timedelta(0)
else:
    follow_time = (datetime.now(timezone.utc) - followers.data[0].followed_at)

over_under = timedelta(days=365)
result = follow_time > over_under

In [104]:
twitch.create_prediction(
    streamer_id, 
    title=f'Has {winner.user_name} been following for over {follow_time}',
    outcomes=['Yes', 'No'],
    prediction_window=60)

<coroutine object Twitch.create_prediction at 0x00000204E2B5F1C0>